## Imports:

In [22]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy as sp

## Constants:

In [2]:
# Maximum alloted time:
MAX_TIME = 24

# Helicopter coordinates:
HELICOPTER = {"x": 0, "y": 0}

# Travelling speed: 
SPEED = 30

## Functions:

In [3]:
def get_to_da_choppa(x, y, helo = HELICOPTER):
    return ((x - helo["x"]) ** 2 + (y - helo["y"]) ** 2) ** 0.5

In [39]:
def travel_time(distance):
    return distance / SPEED

## Setup:

In [5]:
# Getting all the banks:
banks = pd.read_csv("C:/Users/rando/Desktop/data-science/m1-13-project1/bank_data.csv")

# Creating values for new columns:
distance_to_helicopter = []
money_time = []

for i in range(len(banks)):
    distance_to_helicopter.append(get_to_da_choppa(banks.loc[i, "x_coordinate"], banks.loc[i, "y_coordinate"]))
    money_time.append(banks.loc[i, "money"] / banks.loc[i, "time (hr)"])
    
# Inserting new columns:

# Distance to helicopter between each bank
banks.insert(5, "distance_to_chopper", distance_to_helicopter)

# Ratio of money/time
banks.insert(6, "ratio", money_time)

In [36]:
# Option 1 - Richest banks:
rich = banks.sort_values("money", ascending = False).reset_index(drop = True)

# Option 2 - Quickest banks:
quick = banks.sort_values("time (hr)").reset_index(drop = True)

# Option 3 - Closest banks:
close = banks.sort_values("distance_to_chopper").reset_index(drop = True)

# Option 4 - Furthest banks: 
far = banks.sort_values("distance_to_chopper", ascending = False).reset_index(drop = True)

# Option 5 - Best ratio money/time:
ratio = banks.sort_values("ratio", ascending = False).reset_index(drop = True)

# Option 6 - Random!
random = banks.sample(frac = 1).reset_index(drop = True)

# Option 7 - Ratio v 2.0:
ratio2 = ratio.loc[(ratio["money"] > 60000)].reset_index(drop = True)

In [37]:
ratio.insert(7, "distance", 0)
ratio.insert(8, "super ratio", 0)

In [32]:
sp.spatial.distance.squareform(sp.spatial.distance.pdist(ratio.x_coordinate, ratio.y_coordinate))

ValueError: A 2-dimensional array must be passed.

## Algorithm:

In [42]:
def robbery(df):
    
    # results variables
    banks_robbed = []
    total = 0
    time = 0
    
    # next targetted bank, default value is starting bank
    next_bank = df.loc[0, "id"]
    
    # main loop
    for i in range(len(df)):
        
        current_bank = df.loc[i, "id"]
        
        # check if the bank we're looking at is our next target
        if current_bank == next_bank:
            
            #################
            # Current Bank  #
            #################
            
            # distance to helicopter
            x = df.loc[i, "x_coordinate"]
            y = df.loc[i, "y_coordinate"]
            distance = get_to_da_choppa(x, y)

            # travel time to helicopter
            helicopter = travel_time(distance)
            
            # time it would take to rob the bank
            rob = df.loc[i, "time (hr)"]

            # check if current time taken + time to rob + time to travel would land us in jail or not
            if time + helicopter + rob < MAX_TIME:

                time += rob
                banks_robbed.append(current_bank)
                total += df.loc[i, "money"]
            
            #################
            #   Next Bank   #
            #################
            
            # check for next target
            for j in range(i + 1, len(df)):

                # values for the next bank
                next_x = df.loc[j, "x_coordinate"]
                next_y = df.loc[j, "y_coordinate"]
                rob_next = df.loc[j, "time (hr)"]

                # travel time between current and next bank
                distance_between = ((next_x - x) ** 2 + (next_y - y) ** 2) ** 0.5
                travel_next = travel_time(distance_between)

                # travel time between next bank and helicopter
                to_helicopter = get_to_da_choppa(next_x, next_y)
                helicopter_next = travel_time(to_helicopter)

                # check to see if we have enough time for everything
                if time + rob_next + travel_next + helicopter_next < MAX_TIME:
                    
                    time += travel_next
                    next_bank = df.loc[j, "id"]
                    
                    # next target found, break the loop
                    break
            
            # if no next target, return to the helicopter
            if next_bank == current_bank:
                time += helicopter
                
                # no new target, break for loop
                break
            
    return f"You have visited {len(banks_robbed)} banks: {banks_robbed} and stole {total}$ in {time} hours"

## Execution: 

In [53]:
scenes = [
    {"scene": "Option 1: Most valuable banks", "df": rich},
    {"scene": "Option 2: The fastest banks", "df": quick}, 
    {"scene": "Option 3: The closest banks", "df": close}, 
    {"scene": "Option 4: The furthest banks", "df": far},
    {"scene": "Option 5: The best money/time ratio", "df": ratio},
    {"scene": "Option 6: Random order!", "df": random},
    {"scene": "Option 7: Filtered ratio", "df": ratio2},]

for i in range(len(scenes)):
    print(f"{scenes[i]['scene']}\nResults:\n{robbery(scenes[i]['df'])} \n")

Option 1: Most valuable banks
Results:
You have visited 28 banks: [4848, 7265, 8125, 5156, 6439, 4937, 2707, 3061, 5884, 7421, 7963, 6317, 6019, 6333, 8301, 2524, 1447, 1437, 5176, 3008, 8066, 6478, 8841, 6694, 4789, 7760, 6281, 6104] and stole 2835600$ in 23.983292036237216 hours 

Option 2: The fastest banks
Results:
You have visited 128 banks: [8579, 3926, 3613, 3803, 8550, 5725, 1424, 2442, 6528, 9546, 1733, 9583, 4757, 209, 6054, 9195, 4465, 4725, 7343, 4987, 4218, 9529, 7157, 7622, 2194, 3914, 1914, 4364, 8286, 2928, 6401, 4499, 3044, 3798, 5158, 1058, 2932, 5933, 8436, 6566, 4773, 7342, 790, 2161, 6022, 4762, 865, 2827, 2568, 4572, 70, 5732, 9401, 6764, 9241, 524, 6097, 7544, 4274, 1997, 8562, 3466, 2741, 6987, 1611, 8690, 487, 214, 5135, 3582, 8206, 6979, 1874, 2656, 3859, 4828, 2559, 9309, 4605, 6434, 4345, 8022, 5903, 5399, 7106, 4410, 1473, 6234, 4477, 9202, 3150, 7560, 9304, 7434, 9915, 8784, 7258, 58, 8313, 4054, 8966, 5609, 8849, 1222, 9687, 1760, 1757, 2287, 3499, 6468, 

In [46]:
%run check_solution.py

In [49]:
check_solution([4848, 7265, 8125, 7421, 8789], rich)

Time Remaining: 20.563380776130305


9200.0

## Algorithm version 2.0

In [56]:
# not perfected 

def robbery2(df):
    
    # results variables
    banks_robbed = []
    total = 0
    time = 0
    count = 0 
    # always starting on first bank
    current_bank = df.loc[0]
     
    while True:
        count += 1
        print(count)
        #################
        # Current Bank  #
        #################

        # distance to helicopter
        x = current_bank["x_coordinate"]
        y = current_bank["y_coordinate"]
        distance = get_to_da_choppa(x, y)

        # travel time to helicopter
        helicopter = travel_time(distance)

        # time it would take to rob the bank
        rob = current_bank["time (hr)"]

        # check if current time taken + time to rob + time to travel would land us in jail or not
        if time + helicopter + rob < MAX_TIME:

            time += rob
            banks_robbed.append(current_bank["id"])
            total += current_bank["money"]

        #################
        #   Next Bank   #
        #################

        # calculating distance from current bank
        for i in range(len(df)):

            df.loc[i, "distance"] = ((current_bank["x_coordinate"] - current_bank["x_coordinate"]) ** 2 + (current_bank["y_coordinate"] -current_bank["y_coordinate"]) ** 2) ** 0.5
            
        df = df.sort_values("distance").reset_index(drop = True)
        search_radius = df.iloc[:50]

        best_ratio = 0
        next_target = 0
        
        for i in range(len(search_radius)):
            df.loc[i, "super ratio"] = df.loc[i, "money"] / (travel_time(df.loc[i, "distance"]) + df.loc[i, "time (hr)"])
        
        # check for next target
        for i in range(len(search_radius)):

            if search_radius.iloc[i].id not in banks_robbed:

                if search_radius.loc[i, "super ratio"] > best_ratio:

                    best_ratio = search_radius.loc[i, "super ratio"]
                    next_target = i

        target = df.loc[next_target]
        next_x = target["x_coordinate"]
        next_y = target["y_coordinate"]
        rob_next = target["time (hr)"]

        # travel time between current and next bank
        distance_between = ((next_x - x) ** 2 + (next_y - y) ** 2) ** 0.5
        travel_next = travel_time(distance_between)

        # travel time between next bank and helicopter
        to_helicopter = get_to_da_choppa(next_x, next_y)
        helicopter_next = travel_time(to_helicopter)

        # check to see if we have enough time for everything
        if time + rob_next + travel_next + helicopter_next < MAX_TIME:

            time += travel_next
            current_bank = target

        else:
            break

    time += helicopter
                
    return f"You have visited {len(banks_robbed)} banks: {banks_robbed} and stole {total}$ in {time} hours"

In [55]:
robbery2(ratio)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44


'You have visited 44 banks: [3613.0, 3613.0, 9546.0, 3509.0, 3803.0, 4005.0, 6528.0, 4699.0, 9583.0, 6463.0, 5933.0, 7935.0, 9195.0, 1031.0, 4757.0, 5003.0, 8550.0, 9312.0, 4499.0, 3023.0, 9241.0, 5344.0, 7343.0, 8207.0, 3798.0, 1417.0, 7544.0, 7121.0, 4762.0, 9346.0, 8436.0, 3013.0, 8562.0, 512.0, 6097.0, 9507.0, 2928.0, 362.0, 6468.0, 6195.0, 8287.0, 5361.0, 4725.0, 3534.0] and stole 1225100.0$ in 23.919190982333664 hours'